In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip -q train2017.zip

# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !unzip -q annotations_trainval2017.zip

#add val set? enough space in runtime?
#!wget http://images.cocodataset.org/zips/val2017.zip
#!unzip -q val2017.zip

--2021-04-24 22:20:33--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.92.140
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.92.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip.2’

train2017.zip.2       0%[                    ]  81.51M   102MB/s               ^C
replace train2017/000000147328.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import torch
from torchvision import datasets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = datasets.CocoCaptions(
  root="./train2017",
  annFile="./annotations/captions_train2017.json",
)

loading annotations into memory...


FileNotFoundError: ignored

Dataset CocoCaptions
    Number of datapoints: 118287
    Root location: ./train2017

In [ ]:
print('Number of samples: ', len(train_dataset))
img, target = train_dataset[3] # load 4th sample

print("Image Size: ", img.size())
print(target)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
class ColorDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform):
        self.path = path
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(self.path[index]).convert("RGB")
        img = self.transforms(img)
        img = np.array(img)
        img_lab = rgb2lab(img).astype("float32")
        img_lab = transforms.ToTensor()(img_lab)
        L = img_lab[[0], ...] / 100.
        ab = img_lab[[1, 2], ...] / 110.
        
        return {'L': L, 'ab': ab}

    def __len__(self):
        return len(self.path)

In [ ]:
img_transform = transforms.Resize((256, 256),  Image.BICUBIC)

training_dataset = ColorDataset(path="./train2017",transform=img_transform)
training_dataloader = torch.utils.data.DataLoader(dataset, batch_size=16)